# IMPORT

In [1]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import json
import pandas as pd
import numpy as np
import requests
import re
import os
import datetime
from datetime import date, timedelta
from datetime import datetime
import glob
import time 

In [2]:
pd.set_option('display.max_columns', None)

# AUTHENTIFICATION
* guide for authentification with Spotify API: https://developer.spotify.com/documentation/general/guides/authorization-guide/
* calling personal credentials from environment https://able.bio/rhett/how-to-set-and-get-environment-variables-in-python--274rgt5
* open in terminal: touch ~/.bash_profile; open ~/.bash_profile
* implement: source ~/.bash_profile
* get new token https://developer.spotify.com/console/put-playlist/


#### Create and refresh token
run this to  refresh the token. Token runs out every 3600 seconds.

In [39]:
my_client_id=os.getenv('spoti_CLIENT_ID')
my_client_secret=os.getenv('spoti_CLIENT_SECRET')

my_client_id=os.getenv('spoti_CLIENT_ID')
my_client_secret=os.getenv('spoti_CLIENT_SECRET')
sp=spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=my_client_id, client_secret=my_client_secret))


#token to pull songs of an individual usersp_pers=spotipy.Spotify(cache_token_personal)
cache_token_personal= '' 
sp1=spotipy.Spotify(cache_token_personal)


# CRAWLING FOR CHARTS
HERE WE DOWNLOAD CSVs FORM www.spotifycharts.com

### Function
downloads CSVs from spotifycharts.com given the countries of choice and the time interval set below

* Note: this depends on the availability of the CSVs on spoitifycharts.com . Error may occure if we are trying to pull data for a day spotify does not provide charts data for. Slightly adapting frequency or start and end date might shift you around thos "data holes"

In [5]:
#DOWNLOAD LOOP FOR COUNTRY AND FOR TIME PERIOD
# prepare a folder named 'data' with two subfolders named 'viral' and 'regional'
def bulk_charts_download(country_charts, date_list):
    for country in country_charts:
        for date in date_list:
            url= f'https://spotifycharts.com/{what_charts}/{country}/daily/{date}/download'
            r = requests.get(url, allow_redirects=True)
            open(f'data/{what_charts}/{date}-{country}-{what_charts}.csv', 'wb').write(r.content)

### SET PARAMETERS FOR CRAWLING

#### Set Start and End Date

In [6]:
startdate = date(2017, 1, 1)   # start date
enddate = date(2021,1,30)   # end date
delta = enddate - startdate       # as timedelta

date_list=[]

for i in range(delta.days+1):
    day = startdate + timedelta(days=i)
    date_list.append(day.strftime("%Y-%m-%d"))

#### Chose frequency
between start date and end date how often do we pull charts data

In [7]:
date_list=date_list[0::90] #every 90 days

#### Country Codes

In [8]:
#enter list of country codes
country_charts=['at','ar','be','bg','br','ca','ch','cl','co','cr','cz','de','dk','ec','es','fi','fr','gb','gr','is','ie','it','mx','nl','no','pe','pl','pt','ro','ru','se','tr','us']
#HELP: LIST OF COUNTRY CHARTS
print(len(country_charts))
print(sp.country_codes)

33
['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI', 'FR', 'DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']


#### Which charts are we crawling
Options: viral charts or regional top200

In [9]:
# VIRAL CHART sample URL https://spotifycharts.com/viral/us/daily/2020-03-13/download
# thats how i want filename to be safed regional-us-weekly-2020-03-13--2020-03-20.csv

# enter as string 'viral' for viral top 50 or 'regional' for local top 200
what_charts='regional'

#### Trigger download CSVs to folder: Calls function with parameters set above

In [15]:
bulk_charts_download(country_charts, date_list)

# CREATE DATAFRAMES

#### create dataframe from csvs
* intermediate step before adding music features

#### FUNCTION CSVs to Data Frames
* additionally extracts data on country, date and song IDs form the original URLs


In [10]:
def csv_to_dataframe(what_charts):
    path = f'data/{what_charts}/'                     # use your path
    all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

    df = [] # pd.concat takes a list of dataframes as an agrument
    for csv in all_files:
        
        # if condition since csv of top200 look differently than csvs from viral charts.
        # top 200 sheets do have a disclaimer not in the first line that messes with our columns upon import
        # we need to igonre it
        if what_charts=='regional': 
            frame = pd.read_csv(csv,skiprows=[0],error_bad_lines=False, warn_bad_lines=True)
            #frame=frame.drop('Streams',axis=1)
        #this is the read in for viral charts csvs
        else:
            frame = pd.read_csv(csv,skiprows=[],error_bad_lines=False, warn_bad_lines=True)
        
        frame['filename'] = os.path.basename(csv)
        df.append(frame)
        
    #concat list of dataframes into 1
    df=pd.concat(df, ignore_index=True) 
    
    #finetuning columns with regex
    df['ID']= re.findall(r'\d\w+',str(list(df['URL']))) #extract ID from URL
    df['date']=re.findall(r'\d\d\d\d-\d\d-\d\d', str(list(df['filename'])))
    df['date']=pd.to_datetime(df.date, format="%Y-%m-%d", yearfirst=True)
    df['country']=re.findall(r'-[a-z][a-z]-', str(list(df['filename'])))
    df['country']=re.findall(r'[a-z][a-z]',str(list(df['country'])))
    
    # indicates if song is from viral charts or top 200 charts/ 'regional' indicates its part of  regional top200 charts hence we label it 'top200'
    df['group']=np.where(what_charts=='regional','top200',what_charts) 
    return df

In [11]:
charts_df_basic=csv_to_dataframe('regional')#pass in bracket if 'viral' or 'regional'
len(charts_df_basic)

100573

# SUBSETTING TOP200 CHARTS to TOP50
* to make the project feasible for my processing means I need to subset
* my machine is able to make approx 6300 calls to the api per hour
* spotify token expires after 1 hour
* subsetting to the top 20 of 32 countries between 01/01/2017 and 31/12/2020 at 16 sample dates
* ending up with almost 5872 ids to represent 13827 entries

In [32]:
# unique song id: plenty of songs are in teh charts multiple time for different countries and at different dates
# lets see how many IDs are unique since we dont want to pull features for the same song multiple times
unique_ids=list(charts_df_basic.ID.unique())
len(unique_ids)

20585

In [33]:
# lets return all unique IDs of songs that are at one point place in the top200 charts in any country at any time
# and lets check how many unique IDs are left if we subset for only the top 50
#we use this to set the size of our data set
unique_ids_of_top50charts=set(charts_df_basic['ID'][charts_df_basic['ID'].isin (unique_ids) & charts_df_basic['Position'].isin(list(range(1,71)))])
len(unique_ids_of_top50charts)

9082

In [34]:
# using the top 50 of each countries top200 charts at all sample points results in 6982 unique IDs
# thats below above 6300 ongs threashhold - i still give it a shot

In [35]:
#
charts_df_basic_top50=charts_df_basic[charts_df_basic['Position'].isin(list(range(1,71)))]
charts_df_basic_top50.shape 

(36248, 10)

In [36]:
len(charts_df_basic_top50.ID.unique())

9082

#### rename entries in group column from top200 to top50
this is to make subset more clear

In [37]:
charts_df_basic_top50.group=np.where(charts_df_basic_top50.group=='top200','top50',what_charts)

/Users/spitzer/opt/anaconda3/envs/kittycat/lib/python3.8/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# INSPECT DATAFRAME

In [40]:
charts_df_basic_top50.head()

,Position,Track Name,Artist,Streams,URL,filename,ID,date,country,group
0,1,Relación,Sech,76975,https://open.spotify.com/track/3ZG8N7aWw2meb6U...,2020-06-14-pe-regional.csv,3ZG8N7aWw2meb6UrI5ZmnZ,2020-06-14,pe,top50
1,2,Favorito,Camilo,66376,https://open.spotify.com/track/0jT8Nl0shPS8115...,2020-06-14-pe-regional.csv,0jT8Nl0shPS8115is0wD2Q,2020-06-14,pe,top50
2,3,Elegí (feat. Dímelo Flow),Rauw Alejandro,62295,https://open.spotify.com/track/50ZC4PM7hywH27R...,2020-06-14-pe-regional.csv,50ZC4PM7hywH27RcCfViau,2020-06-14,pe,top50
3,4,Safaera,Bad Bunny,55988,https://open.spotify.com/track/2DEZmgHKAvm41k4...,2020-06-14-pe-regional.csv,2DEZmgHKAvm41k4J3R2E9Y,2020-06-14,pe,top50
4,5,Tattoo,Rauw Alejandro,55254,https://open.spotify.com/track/7na7Bk98usp84Fa...,2020-06-14-pe-regional.csv,7na7Bk98usp84FaOJFPv3d,2020-06-14,pe,top50


In [41]:
#are their songs that are in the charts of muliple countries ?
pd.pivot_table(charts_df_basic_top50,index=["Track Name",'Artist'], values=['country','ID'], aggfunc={'country':np.sum,'ID':'count'}).sort_values('ID',ascending=False).head(5)

,,ID,country
Track Name,Artist,,
Shape of You,Ed Sheeran,166,atplischcoieiscapeesczpeptbggbcafrficlbritisec...
Blinding Lights,The Weeknd,141,peiecapltrtrruarromxgrfrptitdkarcrusrobebrclfi...
Dance Monkey,Tones And I,134,peiecocapltrczesrogbbgmxfrptitbrarcrrobeptnode...
Someone You Loved,Lewis Capaldi,112,iecapldenoczesrogbbgptrobeptnodeptroczdkchgbie...
bad guy,Billie Eilish,102,iecocapldenoczrogbbgptrobeptnodeczchplgbiebgtr...


In [42]:
#anything empty?
charts_df_basic_top50.isnull().sum()

Position      0
Track Name    4
Artist        4
Streams       0
URL           0
filename      0
ID            0
date          0
country       0
group         0
dtype: int64

In [43]:
#NAN values
charts_df_basic_top50[charts_df_basic_top50['Artist'].isnull()]

,Position,Track Name,Artist,Streams,URL,filename,ID,date,country,group
31885,22,NaN,NaN,201459,https://open.spotify.com/track/2AgPxySIeAnXDHZ...,2019-06-20-es-regional.csv,2AgPxySIeAnXDHZvAQHg0R,2019-06-20,es,top50
74582,51,NaN,NaN,5066,https://open.spotify.com/track/5cjecvX0CmC9gK0...,2017-12-27-at-regional.csv,5cjecvX0CmC9gK0Laf5EMQ,2017-12-27,at,top50
98935,34,NaN,NaN,113113,https://open.spotify.com/track/5cjecvX0CmC9gK0...,2017-12-27-de-regional.csv,5cjecvX0CmC9gK0Laf5EMQ,2017-12-27,de,top50
98968,67,NaN,NaN,75112,https://open.spotify.com/track/34IY2117PizF4No...,2017-12-27-de-regional.csv,34IY2117PizF4NoRPoMth6,2017-12-27,de,top50


In [44]:
# how many songs per country 
charts_df_basic_top50['country'].value_counts()


co    1190
tr    1190
es    1190
mx    1190
br    1190
ar    1190
pl    1190
it    1190
ca    1190
cl    1190
ie    1190
cz    1190
us    1190
fi    1190
no    1190
cr    1190
pt    1190
se    1190
be    1190
dk    1190
fr    1190
ec    1190
pe    1190
gr    1157
gb    1120
ch    1120
nl    1120
at    1050
de     980
ro     840
is     807
bg     544
ru     140
Name: country, dtype: int64

# CALL API FOR MUSIC FEATURES

### FUNCTION: GET MUSIC FEATURES FROM API

In [45]:
def getTrackFeatures(df):
    feature_names=['ID','duration_ms', 'popularity', 'acousticness','danceability', 'energy', 'key','instrumentalness', 'liveness', 'loudness', 'mode','speechiness', 'valence', 'tempo','loud_start']
    features_df=pd.DataFrame(columns=feature_names)
    
    causing_error=[]
    
    for id in set(df['ID']):#only loop through the unique IDs to enhance performance
        try:
            print(id)
            meta = sp.track(id)
            features = sp.audio_features(id)
            analysis= sp.audio_analysis(id) #does not seem to be available at all songs

            #collect in dictionary
            features_dict = {}   

        
            # meta
            features_dict['ID']=meta['id']
            features_dict['duration_ms'] = meta['duration_ms']
            features_dict['popularity'] = meta['popularity']
            # features
            features_dict['acousticness'] = features[0]['acousticness']
            features_dict['danceability'] = features[0]['danceability']
            features_dict['energy'] = features[0]['energy']
            features_dict['key'] = features[0]['key']
            features_dict['instrumentalness'] = features[0]['instrumentalness']
            features_dict['liveness'] = features[0]['liveness']
            features_dict['loudness'] = features[0]['loudness']
            features_dict['mode'] = features[0]['mode']
            features_dict['speechiness'] = features[0]['speechiness']
            features_dict['valence']=features[0]['valence']
            features_dict['tempo'] = features[0]['tempo']
            #analysis
            features_dict['loud_start']=analysis['sections'][0]['loudness']#loundness at the start
        
        #error handling
        except:
            causing_error.append(id)
            print('causing error: ',causing_error)
            
        #wrap up of all gathered features
        track_df = pd.DataFrame(features_dict,index=[0],columns=features_dict.keys())
        features_df = pd.concat([features_df, track_df], axis=0, ignore_index = True)#concatinating along axis 0
    return features_df
    #lst_columns=['Position','Track Name','Artist','URL','filename','ID','date','country','group','duration_ms', 'popularity', 'acousticness','danceability', 'energy', 'instrumentalness', 'liveness', 'loudness','mode', 'speechiness', 'valence', 'tempo','loud_start']
    #df_with_features=pd.concat([df,features_df],axis=1,ignore_index = True)
    #df_with_features.columns=[list(df.columns)+list(features_df.columns)]#creates new joined list of columns
    #return df_with_features

#### call API on IDs returning a dataframe with features of each ID

In [46]:
#dataframe of features
charts_features_df=getTrackFeatures(charts_df_basic_top50) #get features for each song


0QnEGWGECQUPuE3m8L6hdT
58IL315gMSTD37DOZPJ2hf
6h5PAsRni4IRlxWr6uDPTP
3rOSwuTsUlJp0Pu0MkN8r8
6SfaEpLn8FOVXqWuDdCbCy
5fY9bVl1Wj4PIM3wnduWop
552FgBxuh4rltfhgk7yNZg
0mjbIwkdLPK5g2nGzhacjU
4oMPZbzG1dK2UltYojsKu8
2fqlGvVkoVDCRAA8O5r6bH
1krkFcdC579tuWC334yn1z
71KzugdRW5iyfKUiaABpH1
3NnW2EqJxvHYrRy3upaznl
0OSROf38sIH5vvq33dTTtp
43pg0oNIUwWBCFGZSkomaE
53T4MpkaHLd3oYK5PC5YH2
65NX0qTlnZTDYzTypKfOWD
2FvD20Z8aoWIePi7PoN8sG
263jcrCMb0ttw18hoGQTN4
2E5NGLtMqDmeijWcntP8Ca
3PIDciSFdrQxSQSihim3hN
7C7l4I5OZHEO1Vbx8SGJC2
5MTWw33o2HkA4jHDJCpJky
7apizLyhMRutg3ccXvr2qv
2EaWqg4KUDVx6gVbgQ4Lyw
6ZklqpekeC0KFGzyFVjQuj
6hw1Sy9wZ8UCxYGdpKrU6M
3j30Y6jVypcZxFiWATyZ4Y
1GeGwhJSkJGQLeZ7mdvDdm
0csb0C9E7UGW7Yfi52wGPW
4SIVAJIeJN2KrcL6x5auUH
29R1IMTTbDDA3VNlk6UEW5
4xBjsLUSjcx2h7PcRikpQp
1E3zyoTHtJzsu5CjwIFIhY
43jBqV3j3Xi1g6wO0bhIMd
3YWh4qEPAkWYu2830DFcga
2SBr3MK1sdMs8IxdRcN6qz
6VSaNvPPagk4U5Skxp6XfV
1xKyLu1AbYauopjoAi0bQU
09IStsImFySgyp0pIQdqAc
6whtg5B44HRuvb2zkKrn5D
2KiHXiqQA2dHiLfeSxtTeP
1hEIJTv8PkMXnWTIB2dEYI
0nauWQtOaFK

1nWRwihlQeUoCIjZU3W0Cv
6K5BsR04ijf3FHNzjbaagD
28dgIVVQiTOZlYs3JForb7
7xapw9Oy21WpfEcib2ErSA
4SCM1b1vS70TtNQmO7Lo7B
0nbXyq5TXYPCO7pr3N8S4I
4h6GHYsg2DDAs3j0fA3Ysn
4FQHqppcx6WnsOqJ5QlOjH
5SxkdsY1ufZzoq9iXceLw9
1iIIsJlKQW7E4eLw7x5XOI
3i8RyYJFp3qAJtLMxdgNtf
1TlSTa2eps7lIg6yDjehuT
4fuhLKAuSxRY5OO2hwARlY
2ar6B69vBde0T2n7IGJcff
2cKe57qm50HjR4U4O1aXOb
11P4sSu6BH2lAbWk1sL6cK
6yKkgZBuakFy6SPtsZBR5H
6O85MfGECKBqAK2MgSbCaj
33B8pusNWL0lfl6rm2n8je
7eHHKYb2cRRqqzl8bbiwA3
3bp5fh0qPmCqC5PWEewFkm
0PG9fbaaHFHfre2gUVo7AN
6rfRwqIMFFCDpuvYGvgtpD
768WdBNQJQiDzbMjTZx5aZ
0sXvAOmXgjR2QUqLK1MltU
6ZWsKJdmoYu1oFJbzCmtK9
2BWjJ4Tv26am2QSzwRpQRk
2x67AU9q3lf7cfPSzxCDsL
6VzcQuzTNTMFnJ6rBSaLH9
14eKSIRO006SRXYpfoY8HK
1VEMUkGFio5uURvN8x1DZJ
0EPxmvsG1BY5td4aTOkWBF
24DEtUjFhpV57zGnwzHR4H
7w8XcjYxuykXdg9PTDTjBR
5DtRoFFNian2XZlxRWorpF
4A6V8yOXo7oz6oYhdfHrLl
4rTZbEQtDtlZZW1Ag9Umbk
39S7yxVerin9vMLPsNqHZw
6F2Wg3ENsDNTZtXouMCeef
0poUvBEondVhL0rMukEGEu
14NZoXCrTaI7ClIznTYIpf
2IjdadUTHEJziiFkFMq48t
4IPmigRQtxzdx6IS8gr29K
13ZyrkCDmRz

04CbLJRm3XwOsGZt8mDqbj
5JpUhPdInQTkknuw1wYoA1
1hLkVJtG5XaR18JUvHB5HN
1dAEOpBFYGmQaTke643X2S
3eIRxkHV1xKsSJ8WWFfGjE
7AUhck1sPy11OExQmDDx4P
6hFmSZIw5gmn9mK4hfqozy
6tuzPD1zPPvwMbGiD0TBay
4vKbul7TavBtbzXOqh6njN
7sd72KZS8D59g5NmhxyHpJ
4punHldq7Jfx0Plu7SdjQ0
1Ej96GIBCTvgH7tNX1r3qr
2nhWiRTIwfiN0INbjxkiYZ
1QKbpfbXQP1NY4xy6wpxJA
2wRkBumdItthjYP9XknImg
2jBleFNl3YK8lyJc1OvxXt
46V1YQy4dwmVG9zB5PkawO
1Q8c5YtPFRd79rr4riTQst
5b9tj0yG0Kfn7lWG19uzAH
7mdE30Ov4in9UOCUcQiIK0
5wzhbDTAo8ZM885tHajfTJ
5vqKiYlR4EuMBNlI3dQk4O
5Re3VKVOVnEhUi2TF56dMz
7EjTKUE5HYHIecjIZIYIiW
7zd0UDYFxKL4ymNWKB3eMp
2rbDhOo9Fh61Bbu23T2qCk
7jmyKXzjmjgmloEBZdFIRn
3bBIcLfShA5RAGmlwebWqw
1HxvdVlvQH5NUPi1wsq5L1
6Rg0AJvt1xa4hR2QH2fUhb
6oaXwwoCtIOSCCEHkP9YHF
3FiE2d325mIOc4YjkSLVJR
5h0cywRy1x3LQc7kHIxPgu
3qOxkhNytyb6ifxNzxxULn
2WM1QOmbn4TP0ZFcNOsj2x
04fnJT81xLcyFeWSH5hCQF
5Vw6ZLNB6Ko7VZM6vmNIbr
5bGEezMeTVqGxiciAsOeHJ
7ppLcVgxpoWAhAa2c0CN3b
5KbBGUpIZd6gVFUU8tqYiQ
2r77vqSZkOktTSgmAOoHsV
75VtuVjbcpU9XaAIsh0eGc
3t6j6WNv2OSn6qm5Vxynub
7eJMfftS33K

330QvMk6wENnQy03d9kyCY
0PaNKfwdqHjmW7hVEQ0lbu
032seT1B0E1OdURhzK5Mdj
2XTquzYQAdT1Hk78bOUwsv
5WvAo7DNuPRmk4APhdPzi8
4fE7QQ2f3eah85HWaDGp7M
4OE7BCHElUtM2ATRF7u1PS
4WFSuGah1s5Z6DXYEUFgC2
3GpvoKKWZyTwgbLEM9RmWM
6EFIK4B0jrvLO096QYEhnX
2ALJr1qNX1OgTPBHUGn98v
2KVcqOa2vBKJDIomchdR0C
4Bj3usjRTckEJARO0dLbDf
6eTCxwf1n7XZLucLtFn0Fo
1BkyiAw1L0BOHYckwpRX4V
5fpujciVjGBhnR2XFx06E7
1fyhyOy6iS9E2NckAzjmTo
6kPJZM97LwdG9QIsT7khp6
0YHUITU3w6ws1v8ehT264g
11BrVHgleel7NNoPWdGwgl
4Cy5f4JsH1yoeGks7FnoHw
4aUuOrkNcjyA64ytYqow6o
46v4XAUfknD2BTqZQ60YAX
4kUWA3jkg97wz4bdBgdgHU
0vPVqOKo8LWVganQgBZS5y
0IueCAa7AbmICEdmOpByO1
32lItqlMi4LBhb4k0BaSaC
7Ds48ZCIjmcWWuCeqoDOOO
2huQiKGkgUfp4WjnVRSobL
63iW8lA2sI5dHz6pQg1qH1
4IcysXELq8QqdroY4qEJIi
0Wv5wuenRLI3BcwgT3HPIP
7EiZI6JVHllARrX9PUvAdX
1FuJKBi6XdFyf8Yzmck468
0txHI0BUicGtEOT6FClpiU
5CpkVOosLFc83LbwqO6hnI
2EzRdhuoPyNJDlep7lnNf2
1qfoq1qvZT7VwDVeekCJ97
7MgFU3vP9tF5siiYAyl3VA
2jOhhdNkBc94SfAcrg5Z28
64idia5DEqsgb4z1sSP68t
11EnQRgRMJwMAesfkB5pnu
0V9cosR5jWa4fr2koARmhD
55sNf76dIok

1oGjm7GD7N4BU8CzSgi2Us
1AU7nfy6iBbNt7rrgSglOZ
69Y5IZuBvGyzySFt5x7kCq
2J4P46vCFm1rPkNkp9pZWX
2ZNTZ6YvhoXhoKV7DWOm51
35UvOH0tRlSCdtWtaXuiWh
6c5ghmeB5B3eUwO9Gr4Dgx
20052FYa2SF5Lkp9VZTLJX
1JaBJC3gj7jSsNxsnYTzhj
6qaI3T1s4h3pPAN0qVXwUl
6rQSrBHf7HlZjtcMZ4S4bO
5BUhxai60oAz69BWZv1qW5
5m5rShfBQk4UpG7vpsEKzf
1mVLbFwad1wce7mAhFw2Ih
4F1yvJfQ7gJkrcgFJQDjOr
7oArXBmMlLCtZdUkXWQZGs
1P9oRiZJWnxe2pZmYUZ5eC
2tcTksk6qFbYSaZogGmgfO
53CJANUxooaqGOtdsBTh7O
1cXRnIZuCVSZsdjUlptFiT
1RwoXmQ3RvIdfG8bLeX65g
10nQ3Pj0o1XotcikgJPVKD
0RcrxkcFT5FJpDgaXmzIEF
1MDFp3EvFeoVHiDOMQvGJF
67HY5uY06KnoJnGckIaanZ
3KJ3opyV29269SXkPrc19l
7dQFpbs34ufIEU745DVclf
7yS0sAE8OQkap5iGWimeUd
1lr5prHQn7Q766Ix6eGRKK
3DsLBZjoU5AYxZXsepHtsW
2qqZ5fOGuNvYVzgPFDLQ4l
5y7DaS3jAc9eZzcSOoUIwU
0cvgns9yWdycXStdjLzoZT
3Nc7IorgrSffzaQrQWVw4L
38MpAndIjGAyVdcC6npX95
3XLuehOxBsDJoN3RBO2wfX
5Ru99vS2BaLyBCH1PXFiJo
7FIcIjKaQJpy0vWWFZMZGH
5egD7A5x9AHdVO2fMo3Wbo
1IJH7QjV5V99eJBtKXqGXE
7y7SDLRvb2gNORfBQ7Ouro
7CyCDNO5khwT0eKONyOook
6yc377A8qxHIjPUcjGD4ET
34r4voojt51

...

1zVfFnkHv8ubJRXT30d4T3
0t4v7Q4yzb9SOQohayL33W
0UMLejbZXf7YVgnfxURvVr
0YCMh9HQkRpAtkj0i1sZqH
2CW04trIoYMbroZWDzPAjs
6cFZ4PLC19taNlpl9pbGMf
0pEkK8MqbmGSX7fT8WLMbR
3ILRzR8SwJajRjrcySym2F
5oE4QPgCrAs3itnmj2GmsM
6Aqeqb9rVNA1U6zuuxczQZ
6eqth4FmYzrreFVlTlp53I
5U28RWnN0aRkT2unsxFaOs
70zg99pT51vB4wlMS7e4q7
6rvSAESqueLmDe00pudWtr
3bx3MDkbF2ta2VtLxFxh3w
17Fd6Yb7mSbinKG8LoWfFl
6fwdbPMwP1zVStm8FybmkO
1fobXPCu47gikLMIoyQBLy
2pMCNai7OshHyqb0wwQnO8
2MLxpHtqNLlUmXTkVXai6w
5mfj5WeL3JYtjfExtOXkQ6
52vPpP2gEYaBROIS77J38P
1B9c9f5AzzMu66UOexfRoq
05t7JMip6JrLuSrMV5yYjX
1LIqNVhaIgxXUt2eVoIQJ3
6LnWNqUKhiBgQomH7vWeBX
1nqaSoiLuqevDes4S18sO7
7KJIjFrGYijiBunfFk75oj
2XW4DbS6NddZxRPm5rMCeY
0KkIkfsLEJbrcIhYsCL7L5
19LiotxKSVs7FeVMEdHz63
5lYAphI00oZ5ZtH7CEJDqV
1rExu7075LZrpDfT0Cg8Kk
6lz4EdYV6oWYdAADxOleEz
6puxHBNwu2Nmm7uD3Rd2MP
0B6GRS3vGtzu6w5BQXqWMq
2utt674vADrUfuHtwWZ6hn
0vT797IpZpF5o1XLCnWCUL
1ePxFbp7r2iOkC7W6fGzzO
58N360dQMsyXxgU8pA5vxb
00xE0XQaGE9ScphTFr0PJM
7BuHdm4lQqcyG3qTwdkxus
6D1ExsSlGGexIfqP73Q8Vx
4mkagJJ8AM1

0r1VvoRssNt3yeV8q76SOi
5tblb3Aojs2S91eE1lxWxP
3Nkqp3NSGWMXVVO08zGpN7
0MrZX4ubbrpfweHuHxXUuk
72gemWtPeCOrRnX4PhjNNk
4mTPviBGRgjPxeaQFnBDW3
42CeaId2XNlxugDvyqHfDf
7vdq8RlxxuhB1GFIN9FIlI
38vtTNHApS0p2iDZs9rVQx
4Tvekfes45AT2GWJlqdKar
15coTBAzEN1bOeipoNDZAR
3lTL4LdFKVF51lseI2nk5D
0FDMH2ahuIVzCybwHJK9zh
405klj3hdnIiwvOCQwx3su
48BhFviee97yuYMh4IqMXJ
0YU17F0BlVXvmx5ytsR43w
3ZHnfKXAjulpLqKJfT7DEC
0qA2qykpcGdVl3tiiYum4g
24mpJgP2YZUBif9NvmYyh4
7Kn8yBD9hk8zJtgMQQrTKV
5EKHbW7mXcAR4hCyS3YXjb
2kVHAQKVtczchKctctzbtK
7McVnXcGdaVAzjKpN1BWPb
20HxqjE03spuS8c4mz6HDz
5DKxQmzdX2B0HPYa0s7hNC
75bVc27bDSOAhMAL38cXpV
0E9ZjEAyAwOXZ7wJC0PD33
2NRa651B540evCfhPvwqES
0cDEFCbsDMQbxKodJyRhsK
2x4t2o6x5NVbR62v7BZPHG
3YCvM66mVzyyscuMibOBQI
33bs40fm9vvhv4BITizYUy
6H0AwSQ20mo62jGlPGB8S6
2AHJLtc81zIfj9EKcVEzmJ
146Vb458Y5RbqEa0fYFPzm
0hAfLLkw3l8ZZ8jYsPqbls
7BwEomcUntwQkSWf3U0Yxl
7yYsyXQa2JgbyL9fkEiUNt
5SY5BWTxbDqFouuAdelFV9
6mapJIPnQ23RTAevUoE0DL
0cfqbF7GBQJMYg0S6GEqG4
2lZEnKxY1v6wgR8ipQMwEI
214Mx6MSJhtaO1uPtZGqLH
5pRGXCDY0eT

2FSEIUUQCopkIyBKps8A0E
7nKBxz47S9SD79N086fuhn
0FJeo3KAJ5on89PXmo3FiG
6Ahg1hncxUdK0ICqU03BCu
2XaewhXngZd2Zi4ijFIkK0
75YepABRsjjwo7YLU5QA0X
2S4exmZO3aVEcoxLdKhC47
4wR3kb7ZhEwyy1L5SVhd7C
0dGSOCj6hdV5Nr0V8FjONc
6VTByOxFSK7xt6peo0Rqxg
2yVGgYMwdNIcEdEiUzFUYc
53mrVsi49rLHIaKBiSvElG
59ywHNwwchG4nZJMLyxSzd
4H9vyrlKZpzd1FW3oupnzD
2WR9ohB44rJB2Ak93iwyY2
0X4hMHBj4Ra6h1tIQGl1CH
1GNSWpe2SB30zYGbNTD96b
3eG36tERWexe2lm75Ea3Y8
1KhP9LpqFyzdikipIAmcQx
14wf185UxfNbSy8dwt4r4q
0ZzXPo0gS5h718w7WErG44
4He5TSvX6WhyTeiXpdFaCZ
6BXyD6UUPizpRpA7iOi99r
4wuCQX7JvAZLlrcmH4AeZF
4lzjE9GlGamkuYqQBfCY8R
4z6wo6PJG4Fve45OXK6D9m
4I0R7t0kvVTlqQGpDp0GXl
4yF9erb9bMUIRXr6QdPgP4
5CZbxKM8Ibh3RmfB7xFAzm
3Oc9ywNtQuuutK3ovrYIfe
6eYKSd5QD0Xj45ZDDt47dg
540qZ2h2nFsuky4w42Quw8
3CgZi8DMUftJHK6WL2gKc1
4UHZMWgKDjJ1zS0t6f6qPv
1mT47pKLXUwPYGwB91GcNz
7sO5G9EABYOXQKNPNiE9NR
77a9893yxuz36cBUmBzA8E
5GXAXm5YOmYT0kL5jHvYBt
7AujiK4ndTotwFOaGEjvOu
6Y4PDQv4XjYjHLeLmvyOt0
33JqYIiKMTwSOXZOrn3Rl5
6GhAMqEu0BC8EPYk3YwF7a
3ZROq16fQ1o2dWEGkHPHkJ
0W5Nkn9oyRv

4MEbrM0ASH9jRmoOJQGRUz
64p6ua7zpf66s62StC2QLv
27OWNTEEtJCFVejrh5mqgv
695AOnYHYnvHIMaTNKjkA5
0tXPhc8LvM4dPvoRwI66XQ
03O32oEi0jJsbmcHMjCPBD
1ws9IBGum9LLNEdSVNO2pH
0gn93i7HJt0calph67R5Wu
4pLKXZKr9TsSS5qBQtSIVz
3oIVK7arAFuUOCGafPLxSC
0QFTjMGHWo7joBA5CUlIpd
7aeleTqgkiIZLitnXp9NMl
4mU5iXHeLgbR94siF7p1sY
4yP4P2pHqtejjQ1l12J8bA
4fMjqKYYsvtT6Fvca8lGFl
6AGON2BGdPmPMJGiiNuuwl
1ckLp8lCl8LipXI0ypX72m
3iwZ8hg0q1kTBB5kjg0nO4
1KfrcxnOLyDhd3p4hGrhVe
5bFhY6hwRO5YnOMorbAZAV
5wbELbOJDwV9tU1wZYJdkz
1aZwu6M7UXQxaxpTLCMyhB
2SbIarT6RP85tisGUGzCud
2KzcWcWxIcI8j75Mn2IXkX
4UiaP0wBuy9N9voD91OZOg
70c9sdkTnDiehSDOCxCnDc
52zq2a71SGs3f1vJuv59ui
3rjVpvMgGMj6PRKJgx4nl0
1BxqXAvRM5uFjCQ6lCG328
2turI4o9F4NAZoKstNbUde
2uOLubhbj02hnelPTNRZvV
1HDv4ptf1kBNMzqmlL3JcI
4zq3vo66raGBslISEk6Kg5
2PZILi5FzxEbvYM5agda2T
257tme6AhneIKAqVF1HTH4
4smLSyGtyNJLPb9J1Iuwsb
3u44nNvPgxneJFFIrlbAz2
59sZRCJ3OgYRNvtJxF2srM
1Cml3fnCNLZUC1EEtkcgVb
010Wcwm7AFPRfToQWdmk3l
4eqne0oIdt9GW2ubiliny0
1RvL7eHoeUolnQbWm2Hnba
2WRTnY0slmFgWcrmEr8dPj
6nIoVkKodp1

5A0fbMSRVcJ5grS2gHrVfD
4LQk0DmiAiV5VD0qMgBvSj
6T7auXdjnEFnifaWc796YQ
5k5TKliuhJMQmBXFVMnoCx
4LTHcXXwN8OW4fYm5sVNPP
1504k2wy349EtgioaW78Ht
43cKEIvz1x4gdNvR0bTAoM
3B54sVLJ402zGa6Xm4YGNe
40UroIGvsMPLPBYwH8rMNA
0RyBrDPSh7alcvWdRApQd6
0QqBVoI4u8u6H9ZvJVhJgq
6I8R2dillIEsMrBm6IZqxv
6YzvFfzQNxAUZWYjDOHo6H
62Ev1wG1x0aDOtaVfRolqW
19guFw87TvSkoxnC9EBu2Y
3WR3mQ8SDjJgxbdOFrqfca
63c2jMnVyf8dtnbaQK910d
5KSNUteuNykJDjYiqguIzk
6Vi8rWobVwobca5fR5WHVG
5SYkeNpjhJPFUMOP5TK8iw
4TBp0V4Slb8emBDN8PWU3j
2srL4DYBekshpbprS6H0mO
2n7ohqg1s68ToAHh4u9mcW
2nYpbaLykC2cjU8Yt5KF1v
790MhTXPFrYkrqCKvpg4t1
3NpRkAnxexTh4TklAZNjag
38a7xqi18rg8wGJhK71fj8
4vhkI8x68EaGQR3Kn7Pwhc
6ugYbXPyqfpYTtG9z6YRvH
15OjBeqNX3bShsGYQbteRv
66BjcDhhwWCcQIBxHMpKXh
4kkQGhScJDzNIIGavQE9SN
3b5Li4QKDVBx1x7fQuu54a
3uMB82o0qa7erhiZBW9KQC
3zBm6lRfxl6tlPGbKuaTUn
3YdPhm1aH5fNpjO2nkZCYT
1tdohH49HMAFjlp1QSo6j8
76HlSm9QxyXxpks5HTYS0F
3bWAqKDWg6u1davspr5IkS
7C7GtWZmyicqjBDbAWPsCp
4yjqpsuyrbjyXHB6rC0mWo
1BxkZE73h9BN3qwuA15TA3
3cg2Y9mIiYNdcPTLKaQgK3
5Fla3zyOCcI

In [47]:
print(len(charts_features_df))

# null values left
print(charts_features_df.isnull().sum())
charts_features_df.head()

9082
ID                  0
duration_ms         0
popularity          0
acousticness        0
danceability        0
energy              0
key                 0
instrumentalness    0
liveness            0
loudness            0
mode                0
speechiness         0
valence             0
tempo               0
loud_start          0
dtype: int64


,ID,duration_ms,popularity,acousticness,danceability,energy,key,instrumentalness,liveness,loudness,mode,speechiness,valence,tempo,loud_start
0,0QnEGWGECQUPuE3m8L6hdT,233986,0,0.1930,0.558,0.797,8,0,0.3990,-3.890,0,0.0739,0.368,139.992,-13.082
1,58IL315gMSTD37DOZPJ2hf,234320,0,0.0760,0.899,0.626,6,0,0.0631,-4.228,0,0.2920,0.873,88.007,-10.741
2,6h5PAsRni4IRlxWr6uDPTP,203000,78,0.0275,0.646,0.741,5,0,0.3120,-4.973,0,0.0340,0.571,123.077,-9.706
3,3rOSwuTsUlJp0Pu0MkN8r8,204600,43,0.0102,0.833,0.434,2,0.0219,0.1650,-8.795,1,0.4310,0.281,150.062,-18.515
4,6SfaEpLn8FOVXqWuDdCbCy,219315,51,0.6260,0.744,0.636,11,0,0.3420,-5.293,0,0.0718,0.567,97.527,-15.617


# Leftjoin 
left join df with all charts songs with the dataframe of the unique 6982 music features

In [48]:
charts_df=charts_df_basic_top50.merge(charts_features_df, how='left',on='ID')


In [49]:
#drop rows with missing music data - most notably Baccara "Yes Sir I can boogey"
charts_df.dropna(axis=0, how='any', thresh=17, inplace=True)
len(charts_df)

36206

# EXPORT TO CSV
Analysis will happen in jupyter notebook 'analysis'

In [50]:
charts_df.rename(columns={'length':'duration_ms'})

,Position,Track Name,Artist,Streams,URL,filename,ID,date,country,group,duration_ms,popularity,acousticness,danceability,energy,key,instrumentalness,liveness,loudness,mode,speechiness,valence,tempo,loud_start
0,1,Relación,Sech,76975,https://open.spotify.com/track/3ZG8N7aWw2meb6U...,2020-06-14-pe-regional.csv,3ZG8N7aWw2meb6UrI5ZmnZ,2020-06-14,pe,top50,184254,79,0.1250,0.805,0.562,7,4.96e-06,0.1480,-3.117,0,0.1050,0.767,172.094,-9.440
1,2,Favorito,Camilo,66376,https://open.spotify.com/track/0jT8Nl0shPS8115...,2020-06-14-pe-regional.csv,0jT8Nl0shPS8115is0wD2Q,2020-06-14,pe,top50,209765,75,0.2370,0.606,0.853,9,0,0.2510,-2.975,1,0.0545,0.822,178.043,-11.308
2,3,Elegí (feat. Dímelo Flow),Rauw Alejandro,62295,https://open.spotify.com/track/50ZC4PM7hywH27R...,2020-06-14-pe-regional.csv,50ZC4PM7hywH27RcCfViau,2020-06-14,pe,top50,197721,79,0.2370,0.824,0.631,0,0.000116,0.0531,-2.791,1,0.0809,0.678,171.965,-12.833
3,4,Safaera,Bad Bunny,55988,https://open.spotify.com/track/2DEZmgHKAvm41k4...,2020-06-14-pe-regional.csv,2DEZmgHKAvm41k4J3R2E9Y,2020-06-14,pe,top50,295176,82,0.0103,0.607,0.829,2,0,0.1070,-4.074,1,0.3790,0.685,96.172,-5.742
4,5,Tattoo,Rauw Alejandro,55254,https://open.spotify.com/track/7na7Bk98usp84Fa...,2020-06-14-pe-regional.csv,7na7Bk98usp84FaOJFPv3d,2020-06-14,pe,top50,202886,74,0.0217,0.755,0.617,9,0,0.0625,-5.144,1,0.1820,0.745,193.896,-8.413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36251,66,My Oh My (feat. DaBaby),Camila Cabello,10556,https://open.spotify.com/track/3yOlyBJuViE2YSG...,2020-03-16-fi-regional.csv,3yOlyBJuViE2YSGn3nVE1K,2020-03-16,fi,top50,170746,79,0.0180,0.724,0.491,8,1.29e-05,0.0887,-6.024,1,0.0296,0.383,105.046,-15.027
36252,67,lovely (with Khalid),Billie Eilish,10553,https://open.spotify.com/track/0u2P5u6lvoDfwTY...,2020-03-16-fi-regional.csv,0u2P5u6lvoDfwTYjAADbn4,2020-03-16,fi,top50,200185,87,0.9340,0.351,0.296,4,0,0.0950,-10.109,0,0.0333,0.120,115.284,-27.125
36253,68,Poikien vessa (feat. costee),Mouhous,10530,https://open.spotify.com/track/1NKZmg1pXWa05C0...,2020-03-16-fi-regional.csv,1NKZmg1pXWa05C0yNld9hl,2020-03-16,fi,top50,209305,46,0.0338,0.671,0.776,0,0,0.0907,-4.892,1,0.0556,0.508,131.987,-12.977
36254,69,Señorita,Shawn Mendes,10288,https://open.spotify.com/track/6v3KW9xbzN5yKLt...,2020-03-16-fi-regional.csv,6v3KW9xbzN5yKLt9YKDYA2,2020-03-16,fi,top50,190799,80,0.0392,0.759,0.548,9,0,0.0828,-6.049,0,0.0290,0.749,116.967,-11.510


In [51]:
charts_df.to_csv(r'data/analysis/top50_charts_songs_2017-2020.csv', index = False)

In [52]:
len(charts_df)

36206

In [32]:
###########################################